In [1]:
import numpy as np
import pandas as pd
from boruta import BorutaPy
from sklearn.datasets import load_diabetes
from sklearn.metrics import accuracy_score 
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# Cognition

In [2]:
%cd Z:\PPMI_Data\Excels\NonMotors\Derived250\GeneCpG\Cognition
data = pd.read_csv('CognitionCateg_Methylome_APPRDXFilter.csv')
data.head(1)

Z:\PPMI_Data\Excels\NonMotors\Derived250\GeneCpG\Cognition


,Sentrix,PATNO,HYS,Gender,APPRDX,Montreal_Cognitive,cg08012045,cg08300930,cg00134295,cg10537824,...,cg10054014,cg15364874,cg06622006,cg22071811,cg24960706,cg06917935,cg02794319,cg10702286,cg10353174,cg19590050
0,200991620021_R01C01,3001,2,1,1,0,0.962437,0.964442,0.24353,0.826172,...,0.852622,0.880126,0.876185,0.770531,0.830047,0.216268,0.23435,0.68155,0.103758,0.83789


In [3]:
X = data.drop(['Sentrix','PATNO','HYS','Gender','Montreal_Cognitive','APPRDX'], axis = 1)
y = data['APPRDX']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, random_state = 1) 
print(X.head(2))

   cg08012045  cg08300930  cg00134295  cg10537824  cg04990650  cg03379001  \
0    0.962437    0.964442     0.24353    0.826172    0.845330    0.852316   
1    0.962158    0.963041     0.22606    0.729373    0.830303    0.764861   

   cg02215627  cg06699201  cg18100935  cg19220149  ...  cg10054014  \
0    0.871498    0.653140     0.89167    0.874343  ...    0.852622   
1    0.821899    0.717675     0.88783    0.817058  ...    0.900772   

   cg15364874  cg06622006  cg22071811  cg24960706  cg06917935  cg02794319  \
0    0.880126    0.876185    0.770531    0.830047    0.216268    0.234350   
1    0.901686    0.906222    0.804494    0.905520    0.254265    0.290337   

   cg10702286  cg10353174  cg19590050  
0    0.681550    0.103758    0.837890  
1    0.760554    0.075480    0.833824  

[2 rows x 123017 columns]


In [4]:
rf_all_features = RandomForestClassifier(random_state=1)
rf_all_features.fit(X_train, y_train)

RandomForestClassifier(random_state=1)

In [5]:
accuracy_score(y_test, rf_all_features.predict(X_test))

0.717391304347826

In [7]:
rfc = RandomForestClassifier(random_state=1)
boruta_selector = BorutaPy(rfc, n_estimators= 'auto', verbose=1, random_state=1)
boruta_selector.fit(np.array(X_train), np.array(y_train))  

Iteration: 1 / 100
Iteration: 2 / 100
Iteration: 3 / 100
Iteration: 4 / 100
Iteration: 5 / 100
Iteration: 6 / 100
Iteration: 7 / 100
Iteration: 8 / 100
Iteration: 9 / 100
Iteration: 10 / 100
Iteration: 11 / 100
Iteration: 12 / 100
Iteration: 13 / 100
Iteration: 14 / 100
Iteration: 15 / 100
Iteration: 16 / 100
Iteration: 17 / 100
Iteration: 18 / 100
Iteration: 19 / 100
Iteration: 20 / 100
Iteration: 21 / 100
Iteration: 22 / 100
Iteration: 23 / 100
Iteration: 24 / 100
Iteration: 25 / 100
Iteration: 26 / 100
Iteration: 27 / 100
Iteration: 28 / 100
Iteration: 29 / 100
Iteration: 30 / 100
Iteration: 31 / 100
Iteration: 32 / 100
Iteration: 33 / 100
Iteration: 34 / 100
Iteration: 35 / 100
Iteration: 36 / 100
Iteration: 37 / 100
Iteration: 38 / 100
Iteration: 39 / 100
Iteration: 40 / 100
Iteration: 41 / 100
Iteration: 42 / 100
Iteration: 43 / 100


BorutaPy finished running.

Iteration: 	44 / 100
Confirmed: 	3
Tentative: 	0
Rejected: 	123014


BorutaPy(estimator=RandomForestClassifier(n_estimators=24,
                                          random_state=RandomState(MT19937) at 0x17A4B19ED40),
         n_estimators='auto',
         random_state=RandomState(MT19937) at 0x17A4B19ED40, verbose=1)

In [8]:
# print("Ranking: ",boruta_selector.ranking_)          
print("No. of significant features: ", boruta_selector.n_features_)

No. of significant features:  3


In [9]:
selected_rf_features = pd.DataFrame({'CpG':list(X_train.columns),
                                       'Ranking':boruta_selector.ranking_})
selected_rf_features = selected_rf_features.sort_values(by='Ranking')

In [10]:
Top50 = selected_rf_features.iloc[:51, :]
Top50.head(3)

,CpG,Ranking
11596,cg12345955,1
75799,cg19816560,1
20944,cg12298429,1


In [11]:
Top50.to_csv ('Cognition_Boruta_Top50.csv', index = False)
selected_rf_features.to_csv ('Cognition_Boruta_All.csv', index = False)

In [12]:
# create a list of column names to subset from data
col_names = Top50.iloc[:, 0].tolist()

# subset df based on the column names
df_subset = data.loc[:, data.columns.isin(col_names)]
df_subset['APPRDX'] = data['APPRDX']
df_subset.to_csv ('Cognition_Boruta_Top50_Data.csv', index = False)
df_subset.head(2)

C:\Users\system 4\AppData\Local\Temp\ipykernel_3576\4033399529.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['APPRDX'] = data['APPRDX']


,cg16541194,cg16333036,cg10410319,cg12345955,cg21043558,cg07456278,cg19690051,cg26090534,cg00417288,cg25850998,...,cg04061361,cg22534925,cg12220605,cg12044671,cg13976366,cg16199144,cg16006242,cg25911279,cg07931157,APPRDX
0,0.576279,0.496829,0.021390,0.015210,0.667449,0.582067,0.104141,0.962848,0.892366,0.031865,...,0.568646,0.914514,0.082447,0.869433,0.031021,0.027345,0.020243,0.031463,0.066176,1
1,0.471834,0.463492,0.020285,0.013241,0.659961,0.667340,0.045867,0.963592,0.934470,0.017234,...,0.719066,0.930051,0.113486,0.755008,0.019411,0.033869,0.016203,0.020530,0.069590,1


# Constipation

In [13]:
%cd Z:\PPMI_Data\Excels\NonMotors\Derived250\GeneCpG\Constipation
data = pd.read_csv('ConstipationCateg_Methylome_APPRDXFilter.csv')
data.head(1)

Z:\PPMI_Data\Excels\NonMotors\Derived250\GeneCpG\Constipation


,Sentrix,PATNO,HYS,Gender,APPRDX,MDSP_Constipate,cg13609937,cg13847987,cg13822231,cg20728472,...,cg06819251,cg22158158,cg20161227,cg05701765,cg25437180,cg06002203,cg21722170,cg13142750,cg12743359,cg04293703
0,200991620021_R01C01,3001,2,1,1,0,0.0201,0.922355,0.011956,0.02195,...,0.900701,0.906393,0.948309,0.847871,0.647087,0.836282,0.731078,0.62803,0.883242,0.534427


In [15]:
X = data.drop(['Sentrix','PATNO','HYS','Gender','MDSP_Constipate','APPRDX'], axis = 1)
y = data['APPRDX']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, random_state = 1) 
print(X.head(2))

   cg13609937  cg13847987  cg13822231  cg20728472  cg14567798  cg14205952  \
0    0.020100    0.922355    0.011956    0.021950    0.019629    0.011816   
1    0.071529    0.938088    0.013730    0.029759    0.025837    0.013719   

   cg13083110  cg24453271  cg18015632  cg21195763  ...  cg06819251  \
0    0.531190    0.824585    0.879564    0.805367  ...    0.900701   
1    0.578083    0.820133    0.892031    0.823493  ...    0.739334   

   cg22158158  cg20161227  cg05701765  cg25437180  cg06002203  cg21722170  \
0    0.906393    0.948309    0.847871    0.647087    0.836282    0.731078   
1    0.888118    0.914366    0.854601    0.685887    0.890144    0.816051   

   cg13142750  cg12743359  cg04293703  
0     0.62803    0.883242    0.534427  
1     0.73572    0.892971    0.540466  

[2 rows x 20751 columns]


In [16]:
rf_all_features = RandomForestClassifier(random_state=1)
rf_all_features.fit(X_train, y_train)

RandomForestClassifier(random_state=1)

In [17]:
accuracy_score(y_test, rf_all_features.predict(X_test))

0.7065217391304348

In [18]:
rfc = RandomForestClassifier(random_state=1)
boruta_selector = BorutaPy(rfc, n_estimators= 'auto', verbose=1, random_state=1)
boruta_selector.fit(np.array(X_train), np.array(y_train))  

Iteration: 1 / 100
Iteration: 2 / 100
Iteration: 3 / 100
Iteration: 4 / 100
Iteration: 5 / 100
Iteration: 6 / 100
Iteration: 7 / 100
Iteration: 8 / 100
Iteration: 9 / 100
Iteration: 10 / 100
Iteration: 11 / 100
Iteration: 12 / 100
Iteration: 13 / 100
Iteration: 14 / 100
Iteration: 15 / 100
Iteration: 16 / 100
Iteration: 17 / 100
Iteration: 18 / 100
Iteration: 19 / 100
Iteration: 20 / 100
Iteration: 21 / 100
Iteration: 22 / 100
Iteration: 23 / 100
Iteration: 24 / 100
Iteration: 25 / 100
Iteration: 26 / 100
Iteration: 27 / 100
Iteration: 28 / 100
Iteration: 29 / 100
Iteration: 30 / 100
Iteration: 31 / 100
Iteration: 32 / 100
Iteration: 33 / 100
Iteration: 34 / 100
Iteration: 35 / 100
Iteration: 36 / 100
Iteration: 37 / 100
Iteration: 38 / 100
Iteration: 39 / 100
Iteration: 40 / 100
Iteration: 41 / 100
Iteration: 42 / 100
Iteration: 43 / 100
Iteration: 44 / 100
Iteration: 45 / 100
Iteration: 46 / 100


BorutaPy finished running.

Iteration: 	47 / 100
Confirmed: 	1
Tentative: 	0
Rejected: 

BorutaPy(estimator=RandomForestClassifier(n_estimators=14,
                                          random_state=RandomState(MT19937) at 0x17A0532B340),
         n_estimators='auto',
         random_state=RandomState(MT19937) at 0x17A0532B340, verbose=1)

In [19]:
# print("Ranking: ",boruta_selector.ranking_)          
print("No. of significant features: ", boruta_selector.n_features_)

No. of significant features:  1


In [20]:
selected_rf_features = pd.DataFrame({'CpG':list(X_train.columns),
                                       'Ranking':boruta_selector.ranking_})
selected_rf_features = selected_rf_features.sort_values(by='Ranking')

In [21]:
Top50 = selected_rf_features.iloc[:51, :]
Top50.head(3)

,CpG,Ranking
3459,cg12298429,1
3545,cg27281389,2
14151,cg17528967,3


In [22]:
Top50.to_csv ('Constipation_Boruta_Top50.csv', index = False)
selected_rf_features.to_csv ('Constipation_Boruta_All.csv', index = False)

In [23]:
# create a list of column names to subset from data
col_names = Top50.iloc[:, 0].tolist()

# subset df based on the column names
df_subset = data.loc[:, data.columns.isin(col_names)]
df_subset['APPRDX'] = data['APPRDX']
df_subset.to_csv ('Constipation_Boruta_Top50_Data.csv', index = False)
df_subset.head(2)

C:\Users\system 4\AppData\Local\Temp\ipykernel_3576\3466572124.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['APPRDX'] = data['APPRDX']


,cg22237213,cg06751221,cg13332222,cg14005240,cg14889643,cg17612987,cg21043558,cg12298429,cg27281389,cg25434660,...,cg20187858,cg09233704,cg05142211,cg12683454,cg09043524,cg14196331,cg26458329,cg19371224,cg02454595,APPRDX
0,0.922623,0.057391,0.902238,0.878486,0.017995,0.031849,0.667449,0.032801,0.026921,0.848560,...,0.164275,0.620103,0.035018,0.023826,0.926060,0.034453,0.029374,0.751572,0.058504,1
1,0.918179,0.083624,0.928699,0.887782,0.020710,0.031061,0.659961,0.042715,0.033420,0.864457,...,0.122112,0.682452,0.041906,0.028970,0.898801,0.028081,0.021559,0.855625,0.042324,1


# Depression

In [24]:
%cd Z:\PPMI_Data\Excels\NonMotors\Derived250\GeneCpG\Depression
data = pd.read_csv('DepressionCateg_Methylome_APPRDXFilter.csv')
data.head(1)

Z:\PPMI_Data\Excels\NonMotors\Derived250\GeneCpG\Depression


,Sentrix,PATNO,HYS,Gender,APPRDX,MDS_Depress,cg20815302,cg25795414,cg00959375,cg07781847,...,cg02293835,cg25106148,cg09135009,cg16616325,cg08020573,cg19102621,cg26302779,cg10377074,cg06750710,ch.14.849914F
0,200991620021_R01C01,3001,2,1,1,0,0.907957,0.86891,0.752844,0.827413,...,0.890264,0.096523,0.088376,0.035056,0.040317,0.620467,0.917245,0.87378,0.809904,0.045885


In [25]:
X = data.drop(['Sentrix','PATNO','HYS','Gender','MDS_Depress','APPRDX'], axis = 1)
y = data['APPRDX']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, random_state = 1) 
print(X.head(2))

   cg20815302  cg25795414  cg00959375  cg07781847  cg05138383  cg27394794  \
0    0.907957    0.868910    0.752844    0.827413    0.771393    0.766283   
1    0.918626    0.865257    0.798170    0.823846    0.760193    0.801117   

   cg12443990  cg19788329  cg22178513  cg04167299  ...  cg02293835  \
0    0.791932    0.785104    0.723989    0.861925  ...    0.890264   
1    0.772456    0.742722    0.580721    0.898379  ...    0.841768   

   cg25106148  cg09135009  cg16616325  cg08020573  cg19102621  cg26302779  \
0    0.096523    0.088376    0.035056    0.040317    0.620467    0.917245   
1    0.056917    0.062589    0.032271    0.033286    0.656780    0.876781   

   cg10377074  cg06750710  ch.14.849914F  
0    0.873780    0.809904       0.045885  
1    0.891459    0.791871       0.061687  

[2 rows x 106975 columns]


In [26]:
rf_all_features = RandomForestClassifier(random_state=1)
rf_all_features.fit(X_train, y_train)

RandomForestClassifier(random_state=1)

In [27]:
accuracy_score(y_test, rf_all_features.predict(X_test))

0.7065217391304348

In [28]:
rfc = RandomForestClassifier(random_state=1)
boruta_selector = BorutaPy(rfc, n_estimators= 'auto', verbose=1, random_state=1)
boruta_selector.fit(np.array(X_train), np.array(y_train))  

Iteration: 1 / 100
Iteration: 2 / 100
Iteration: 3 / 100
Iteration: 4 / 100
Iteration: 5 / 100
Iteration: 6 / 100
Iteration: 7 / 100
Iteration: 8 / 100
Iteration: 9 / 100
Iteration: 10 / 100
Iteration: 11 / 100
Iteration: 12 / 100
Iteration: 13 / 100
Iteration: 14 / 100
Iteration: 15 / 100
Iteration: 16 / 100
Iteration: 17 / 100
Iteration: 18 / 100
Iteration: 19 / 100
Iteration: 20 / 100
Iteration: 21 / 100
Iteration: 22 / 100
Iteration: 23 / 100
Iteration: 24 / 100
Iteration: 25 / 100
Iteration: 26 / 100
Iteration: 27 / 100
Iteration: 28 / 100
Iteration: 29 / 100
Iteration: 30 / 100
Iteration: 31 / 100
Iteration: 32 / 100
Iteration: 33 / 100
Iteration: 34 / 100
Iteration: 35 / 100
Iteration: 36 / 100
Iteration: 37 / 100
Iteration: 38 / 100
Iteration: 39 / 100
Iteration: 40 / 100
Iteration: 41 / 100
Iteration: 42 / 100
Iteration: 43 / 100
Iteration: 44 / 100
Iteration: 45 / 100
Iteration: 46 / 100


BorutaPy finished running.

Iteration: 	47 / 100
Confirmed: 	2
Tentative: 	0
Rejected: 

BorutaPy(estimator=RandomForestClassifier(n_estimators=20,
                                          random_state=RandomState(MT19937) at 0x17A47D8D040),
         n_estimators='auto',
         random_state=RandomState(MT19937) at 0x17A47D8D040, verbose=1)

In [29]:
# print("Ranking: ",boruta_selector.ranking_)          
print("No. of significant features: ", boruta_selector.n_features_)

No. of significant features:  2


In [30]:
selected_rf_features = pd.DataFrame({'CpG':list(X_train.columns),
                                       'Ranking':boruta_selector.ranking_})
selected_rf_features = selected_rf_features.sort_values(by='Ranking')

In [31]:
Top50 = selected_rf_features.iloc[:51, :]
Top50.head(3)

,CpG,Ranking
25511,cg21092373,1
105610,cg11485463,1
56467,cg24094897,2


In [32]:
Top50.to_csv ('Depression_Boruta_Top50.csv', index = False)
selected_rf_features.to_csv ('Depression_Boruta_All.csv', index = False)

In [33]:
# create a list of column names to subset from data
col_names = Top50.iloc[:, 0].tolist()

# subset df based on the column names
df_subset = data.loc[:, data.columns.isin(col_names)]
df_subset['APPRDX'] = data['APPRDX']
df_subset.to_csv ('Depression_Boruta_Top50_Data.csv', index = False)
df_subset.head(2)

C:\Users\system 4\AppData\Local\Temp\ipykernel_3576\842568119.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['APPRDX'] = data['APPRDX']


,cg18780217,cg21747188,cg04841205,cg00052836,cg07456278,cg12510028,cg19690051,cg21062891,cg21096050,cg12298429,...,cg12220605,cg12683454,cg13976366,cg25911279,cg19371224,cg12600134,cg11485463,cg23026864,cg19928377.1,APPRDX
0,0.038530,0.064156,0.729990,0.872534,0.582067,0.029810,0.104141,0.887982,0.893820,0.032801,...,0.082447,0.023826,0.031021,0.031463,0.751572,0.025254,0.011261,0.074048,0.649511,1
1,0.022502,0.056883,0.790912,0.916062,0.667340,0.035511,0.045867,0.923072,0.884377,0.042715,...,0.113486,0.028970,0.019411,0.020530,0.855625,0.017894,0.013494,0.091529,0.686080,1


# Fatigue

In [34]:
%cd Z:\PPMI_Data\Excels\NonMotors\Derived250\GeneCpG\Fatigue
data = pd.read_csv('FatigueCateg_Methylome_APPRDXFilter.csv')
data.head(1)

Z:\PPMI_Data\Excels\NonMotors\Derived250\GeneCpG\Fatigue


,Sentrix,PATNO,HYS,Gender,APPRDX,MDSP_Fatigue,cg17960051,cg06562372,cg04099454,cg18542829,...,cg15500865,cg10329418,cg18241308,cg04350520,cg04685170,cg20985067,cg08461772,cg25879118,cg07510163,ch.13.693633F
0,200991620021_R01C01,3001,2,1,1,1,0.019716,0.024434,0.895146,0.024184,...,0.105174,0.262518,0.784256,0.886425,0.080717,0.814912,0.331552,0.916387,0.841837,0.036872


In [35]:
X = data.drop(['Sentrix','PATNO','HYS','Gender','MDSP_Fatigue','APPRDX'], axis = 1)
y = data['APPRDX']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, random_state = 1) 
print(X.head(2))

   cg17960051  cg06562372  cg04099454  cg18542829  cg04599912  cg15759056  \
0    0.019716    0.024434    0.895146    0.024184    0.014927    0.012150   
1    0.020417    0.016055    0.882493    0.045616    0.021537    0.013626   

   cg16703576  cg00031165  cg26835636  cg15446670  ...  cg15500865  \
0    0.026345    0.078477    0.016245    0.062766  ...    0.105174   
1    0.032826    0.033160    0.018156    0.093692  ...    0.066103   

   cg10329418  cg18241308  cg04350520  cg04685170  cg20985067  cg08461772  \
0    0.262518    0.784256    0.886425    0.080717    0.814912    0.331552   
1    0.210723    0.905081    0.882225    0.082907    0.906400    0.497660   

   cg25879118  cg07510163  ch.13.693633F  
0    0.916387    0.841837       0.036872  
1    0.915460    0.822889       0.039866  

[2 rows x 31211 columns]


In [36]:
rf_all_features = RandomForestClassifier(random_state=1)
rf_all_features.fit(X_train, y_train)

RandomForestClassifier(random_state=1)

In [37]:
accuracy_score(y_test, rf_all_features.predict(X_test))

0.7065217391304348

In [38]:
rfc = RandomForestClassifier(random_state=1)
boruta_selector = BorutaPy(rfc, n_estimators= 'auto', verbose=1, random_state=1)
boruta_selector.fit(np.array(X_train), np.array(y_train))  

Iteration: 1 / 100
Iteration: 2 / 100
Iteration: 3 / 100
Iteration: 4 / 100
Iteration: 5 / 100
Iteration: 6 / 100
Iteration: 7 / 100
Iteration: 8 / 100


BorutaPy finished running.

Iteration: 	9 / 100
Confirmed: 	0
Tentative: 	0
Rejected: 	31211


BorutaPy(estimator=RandomForestClassifier(n_estimators=2498,
                                          random_state=RandomState(MT19937) at 0x17A47D8D140),
         n_estimators='auto',
         random_state=RandomState(MT19937) at 0x17A47D8D140, verbose=1)

In [39]:
# print("Ranking: ",boruta_selector.ranking_)          
print("No. of significant features: ", boruta_selector.n_features_)

No. of significant features:  0


In [40]:
selected_rf_features = pd.DataFrame({'CpG':list(X_train.columns),
                                       'Ranking':boruta_selector.ranking_})
selected_rf_features = selected_rf_features.sort_values(by='Ranking')

In [41]:
Top50 = selected_rf_features.iloc[:51, :]
Top50.head(3)

,CpG,Ranking
10169,cg23668004,2
14758,cg09197492,3
21009,cg11036438,4


In [42]:
Top50.to_csv ('Fatigue_Boruta_Top50.csv', index = False)
selected_rf_features.to_csv ('Fatigue_Boruta_All.csv', index = False)

In [43]:
# create a list of column names to subset from data
col_names = Top50.iloc[:, 0].tolist()

# subset df based on the column names
df_subset = data.loc[:, data.columns.isin(col_names)]
df_subset['APPRDX'] = data['APPRDX']
df_subset.to_csv ('Fatigue_Boruta_Top50_Data.csv', index = False)
df_subset.head(2)

C:\Users\system 4\AppData\Local\Temp\ipykernel_3576\2639640420.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['APPRDX'] = data['APPRDX']


,cg06751221,cg03476938,cg23655939,cg21043558,cg19226007,cg18473117,cg12510028,cg27133496,cg17422176,cg01020980,...,cg09233704,cg22749855,cg05142211,cg00883689,cg12220605,cg12683454,cg16494828,cg00651709,cg15267345,APPRDX
0,0.057391,0.549004,0.515701,0.667449,0.040020,0.938774,0.029810,0.510178,0.117596,0.011566,...,0.620103,0.843622,0.035018,0.525123,0.082447,0.023826,0.012698,0.899361,0.029328,1
1,0.083624,0.561954,0.523438,0.659961,0.017811,0.928175,0.035511,0.653650,0.147173,0.011167,...,0.682452,0.816596,0.041906,0.578082,0.113486,0.028970,0.012574,0.897422,0.026110,1


# SymbolDigit  

In [70]:
%cd Z:\PPMI_Data\Excels\NonMotors\Derived250\GeneCpG\SymbolDigit
data = pd.read_csv('SymbDigiCateg_Methylome_APPRDXFilter.csv')
data.head(1)

Z:\PPMI_Data\Excels\NonMotors\Derived250\GeneCpG\SymbolDigit


,Sentrix,PATNO,HYS,Gender,APPRDX,Symbol_Digit,cg20815302,cg25795414,cg00959375,cg07781847,...,cg03814399,cg14792798,cg16910097,cg08750857,cg17933893,cg06917935,cg02794319,cg10702286,cg10353174,cg19590050
0,200991620021_R01C01,3001,2,1,1,1,0.907957,0.86891,0.752844,0.827413,...,0.880496,0.928546,0.900567,0.8778,0.836823,0.216268,0.23435,0.68155,0.103758,0.83789


In [71]:
X = data.drop(['Sentrix','PATNO','HYS','Gender','Symbol_Digit','APPRDX'], axis = 1)
y = data['APPRDX']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, random_state = 1) 
print(X.head(2))

   cg20815302  cg25795414  cg00959375  cg07781847  cg05138383  cg27394794  \
0    0.907957    0.868910    0.752844    0.827413    0.771393    0.766283   
1    0.918626    0.865257    0.798170    0.823846    0.760193    0.801117   

   cg12443990  cg19788329  cg22178513  cg04167299  ...  cg03814399  \
0    0.791932    0.785104    0.723989    0.861925  ...    0.880496   
1    0.772456    0.742722    0.580721    0.898379  ...    0.877306   

   cg14792798  cg16910097  cg08750857  cg17933893  cg06917935  cg02794319  \
0    0.928546    0.900567    0.877800    0.836823    0.216268    0.234350   
1    0.912956    0.935591    0.942104    0.750613    0.254265    0.290337   

   cg10702286  cg10353174  cg19590050  
0    0.681550    0.103758    0.837890  
1    0.760554    0.075480    0.833824  

[2 rows x 192582 columns]


In [72]:
rf_all_features = RandomForestClassifier(random_state=1)
rf_all_features.fit(X_train, y_train)

RandomForestClassifier(random_state=1)

In [73]:
accuracy_score(y_test, rf_all_features.predict(X_test))

0.6956521739130435

In [74]:
rfc = RandomForestClassifier(random_state=1)
boruta_selector = BorutaPy(rfc, n_estimators= 'auto', verbose=1, random_state=1)
boruta_selector.fit(np.array(X_train), np.array(y_train))  

Iteration: 1 / 100
Iteration: 2 / 100
Iteration: 3 / 100
Iteration: 4 / 100
Iteration: 5 / 100
Iteration: 6 / 100
Iteration: 7 / 100
Iteration: 8 / 100
Iteration: 9 / 100
Iteration: 10 / 100
Iteration: 11 / 100
Iteration: 12 / 100
Iteration: 13 / 100
Iteration: 14 / 100
Iteration: 15 / 100
Iteration: 16 / 100
Iteration: 17 / 100
Iteration: 18 / 100
Iteration: 19 / 100
Iteration: 20 / 100
Iteration: 21 / 100
Iteration: 22 / 100
Iteration: 23 / 100
Iteration: 24 / 100
Iteration: 25 / 100
Iteration: 26 / 100
Iteration: 27 / 100
Iteration: 28 / 100
Iteration: 29 / 100
Iteration: 30 / 100
Iteration: 31 / 100
Iteration: 32 / 100
Iteration: 33 / 100
Iteration: 34 / 100


BorutaPy finished running.

Iteration: 	35 / 100
Confirmed: 	1
Tentative: 	0
Rejected: 	192581


BorutaPy(estimator=RandomForestClassifier(n_estimators=14,
                                          random_state=RandomState(MT19937) at 0x17A4C5D2740),
         n_estimators='auto',
         random_state=RandomState(MT19937) at 0x17A4C5D2740, verbose=1)

In [75]:
# print("Ranking: ",boruta_selector.ranking_)          
print("No. of significant features: ", boruta_selector.n_features_)

No. of significant features:  1


In [76]:
selected_rf_features = pd.DataFrame({'CpG':list(X_train.columns),
                                       'Ranking':boruta_selector.ranking_})
selected_rf_features = selected_rf_features.sort_values(by='Ranking')

In [77]:
Top50 = selected_rf_features.iloc[:51, :]
Top50.head(3)

,CpG,Ranking
129347,cg16078938,1
117259,cg19816560,2
44329,cg21092373,3


In [78]:
Top50.to_csv ('SymbDigi_Boruta_Top50.csv', index = False)
selected_rf_features.to_csv ('SymbDigi_Boruta_All.csv', index = False)

In [79]:
# create a list of column names to subset from data
col_names = Top50.iloc[:, 0].tolist()

# subset df based on the column names
df_subset = data.loc[:, data.columns.isin(col_names)]
df_subset['APPRDX'] = data['APPRDX']
df_subset.to_csv ('SymbDigi_Boruta_Top50_Data.csv', index = False)
df_subset.head(2)

C:\Users\system 4\AppData\Local\Temp\ipykernel_3576\275907179.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['APPRDX'] = data['APPRDX']


,cg20250922,cg16541194,cg21747188,cg10410319,cg12345955,cg19690051,cg21062891,cg00417288,cg12298429,cg14648916,...,cg22534925,cg12179916,cg12044671,cg19555872,cg25273343,cg04286615,cg07931157,cg01120173,cg12600134,APPRDX
0,0.279845,0.576279,0.064156,0.021390,0.015210,0.104141,0.887982,0.892366,0.032801,0.014798,...,0.914514,0.638415,0.869433,0.879701,0.736708,0.034118,0.066176,0.10021,0.025254,1
1,0.241012,0.471834,0.056883,0.020285,0.013241,0.045867,0.923072,0.934470,0.042715,0.019783,...,0.930051,0.679014,0.755008,0.894245,0.876514,0.020293,0.069590,0.16000,0.017894,1


# UPSIT  

In [2]:
%cd Z:\PPMI_Data\Excels\NonMotors\Derived250\GeneCpG\UPSIT
data = pd.read_csv('UPSIT3Categ_Methylome_APPRDXFilter.csv')
data.head(1)

Z:\PPMI_Data\Excels\NonMotors\Derived250\GeneCpG\UPSIT


,Sentrix,PATNO,HYS,Gender,APPRDX,UPSIT_3,UPSIT_Total,cg06067658,cg15442538,cg25928446,...,cg05766881,cg01708543,cg10933281,cg04735449,cg15856662,cg11864127,cg14420798,cg08859349,cg01481205,cg19228755
0,200991620021_R01C01,3001,2,1,1,1,25,0.088679,0.890148,0.719238,...,0.748763,0.830052,0.861102,0.855659,0.788271,0.805084,0.729165,0.803183,0.925666,0.849608


In [3]:
X = data.drop(['Sentrix','PATNO','HYS','Gender','UPSIT_3','UPSIT_Total','APPRDX'], axis = 1)
y = data['APPRDX']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, random_state = 1) 
print(X.head(2))

   cg06067658  cg15442538  cg25928446  cg12415575  cg26358225  cg19058526  \
0    0.088679    0.890148    0.719238    0.066678    0.151532    0.133249   
1    0.128668    0.916448    0.726893    0.066768    0.198886    0.141136   

   cg20195763  cg23725394  cg27107076  cg15657055  ...  cg05766881  \
0    0.479724    0.926745    0.538423    0.697562  ...    0.748763   
1    0.412058    0.876792    0.665097    0.797354  ...    0.886279   

   cg01708543  cg10933281  cg04735449  cg15856662  cg11864127  cg14420798  \
0    0.830052    0.861102    0.855659    0.788271    0.805084    0.729165   
1    0.898054    0.896800    0.911732    0.863749    0.893595    0.829444   

   cg08859349  cg01481205  cg19228755  
0    0.803183    0.925666    0.849608  
1    0.866504    0.917233    0.786606  

[2 rows x 5672 columns]


In [4]:
rf_all_features = RandomForestClassifier(random_state=1)
rf_all_features.fit(X_train, y_train)

RandomForestClassifier(random_state=1)

In [5]:
accuracy_score(y_test, rf_all_features.predict(X_test))

0.717391304347826

In [7]:
rfc = RandomForestClassifier(random_state=1)
boruta_selector = BorutaPy(rfc, n_estimators= 'auto', verbose=1, random_state=1)
boruta_selector.fit(np.array(X_train), np.array(y_train))  

Iteration: 1 / 100
Iteration: 2 / 100
Iteration: 3 / 100
Iteration: 4 / 100
Iteration: 5 / 100
Iteration: 6 / 100
Iteration: 7 / 100
Iteration: 8 / 100
Iteration: 9 / 100
Iteration: 10 / 100
Iteration: 11 / 100
Iteration: 12 / 100
Iteration: 13 / 100
Iteration: 14 / 100
Iteration: 15 / 100
Iteration: 16 / 100
Iteration: 17 / 100
Iteration: 18 / 100
Iteration: 19 / 100
Iteration: 20 / 100
Iteration: 21 / 100
Iteration: 22 / 100
Iteration: 23 / 100
Iteration: 24 / 100
Iteration: 25 / 100
Iteration: 26 / 100
Iteration: 27 / 100
Iteration: 28 / 100
Iteration: 29 / 100
Iteration: 30 / 100
Iteration: 31 / 100
Iteration: 32 / 100


BorutaPy finished running.

Iteration: 	33 / 100
Confirmed: 	1
Tentative: 	0
Rejected: 	5671


BorutaPy(estimator=RandomForestClassifier(n_estimators=14,
                                          random_state=RandomState(MT19937) at 0x1F4D6438940),
         n_estimators='auto',
         random_state=RandomState(MT19937) at 0x1F4D6438940, verbose=1)

In [8]:
# print("Ranking: ",boruta_selector.ranking_)          
print("No. of significant features: ", boruta_selector.n_features_)

No. of significant features:  1


In [9]:
selected_rf_features = pd.DataFrame({'CpG':list(X_train.columns),
                                       'Ranking':boruta_selector.ranking_})
selected_rf_features = selected_rf_features.sort_values(by='Ranking')

In [10]:
Top50 = selected_rf_features.iloc[:51, :]
Top50.head(3)

,CpG,Ranking
883,cg02152968,1
5204,cg05142211,2
3939,cg26484715,3


In [11]:
Top50.to_csv ('UPSIT_Boruta_Top50.csv', index = False)
selected_rf_features.to_csv ('UPSIT_Boruta_All.csv', index = False)

In [12]:
# create a list of column names to subset from data
col_names = Top50.iloc[:, 0].tolist()

# subset df based on the column names
df_subset = data.loc[:, data.columns.isin(col_names)]
df_subset['PATNO'] = data['PATNO']
df_subset['APPRDX'] = data['APPRDX']
df_subset.to_csv ('UPSIT_Boruta_Top50_Data.csv', index = False)
df_subset.head(2)

C:\Users\system 4\AppData\Local\Temp\ipykernel_1284\4138935079.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['PATNO'] = data['PATNO']
C:\Users\system 4\AppData\Local\Temp\ipykernel_1284\4138935079.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['APPRDX'] = data['APPRDX']


,cg14719675,cg00260798,cg10244356,cg00877792,cg12534008,cg03476938,cg10061292,cg23205529,cg23074747,cg02152968,...,cg01117061,cg08125401,cg07897734,cg06632027,cg00538060,cg05142211,cg13959611,cg19458274,PATNO,APPRDX
0,0.043587,0.885679,0.592379,0.497734,0.253873,0.549004,0.013281,0.054117,0.011630,0.263403,...,0.024752,0.136334,0.842938,0.029793,0.785801,0.035018,0.044262,0.018518,3001,1
1,0.080217,0.916535,0.609867,0.453499,0.229244,0.561954,0.012520,0.048337,0.011863,0.231128,...,0.080059,0.081183,0.923446,0.027356,0.818536,0.041906,0.033433,0.019884,3007,1
